In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *


## Çok Dönemli İş  Planlaması 

In [7]:
değişkenler=[]
for ay in ["Ocak","Şubat","Mart","Nisan","Mayıs","Haziran"]:
        for opsiyon in ["1 aylık","2 aylık","3 aylık"]:
            değişkenler.append((ay,opsiyon))

In [8]:
değişkenler

[('Ocak', '1 aylık'),
 ('Ocak', '2 aylık'),
 ('Ocak', '3 aylık'),
 ('Şubat', '1 aylık'),
 ('Şubat', '2 aylık'),
 ('Şubat', '3 aylık'),
 ('Mart', '1 aylık'),
 ('Mart', '2 aylık'),
 ('Mart', '3 aylık'),
 ('Nisan', '1 aylık'),
 ('Nisan', '2 aylık'),
 ('Nisan', '3 aylık'),
 ('Mayıs', '1 aylık'),
 ('Mayıs', '2 aylık'),
 ('Mayıs', '3 aylık'),
 ('Haziran', '1 aylık'),
 ('Haziran', '2 aylık'),
 ('Haziran', '3 aylık')]

In [ ]:
def model_oluştur():
    model = ConcreteModel()
    bilgisayar_talep=[9,5,7,9,10,5]
    maliyet_1_aylık=200
    maliyet_2_aylık=350
    maliyet_3_aylık=450
    maliyet_2_aylık_uzatma=175
    maliyet_3_aylık_uzatma=150
    
    değişkenler=[]
    for ay in ["Ocak","Şubat","Mart","Nisan","Mayıs","Haziran"]:
            for opsiyon in ["1 aylık","2 aylık","3 aylık"]:
                değişkenler.append((ay,opsiyon))
    for ay in ["Mart","Nisan","Mayıs","Haziran"]:
         değişkenler.append((ay,"2 aylık uzatım"))
    for ay in ["Nisan","Mayıs","Haziran"]:
         değişkenler.append((ay,"3 aylık uzatım"))
    
    

In [6]:
[i for i in range(6)]

[0, 1, 2, 3, 4, 5]

In [ ]:
process_variables={
    1:{'time':2,'cost':2,'output':{'input_1':2},'input':{'raw':1}},
    2:{'time':2,'cost':4,'output':{'input_2':3},'input':{'raw':1}},
    3:{'time':2,'cost':1,'output':{'product_A':1,'waste':1},'input':{'input_1':2,'input_2':1}},
    4:{'time':3,'cost':8,'output':{'product_B':1,'waste':.8},'input':{'input_1':1,'input_2':2}},
     5:{'time':1,'cost':4,'output':{'product_C':1},'input':{'input_1':2, 'waste':.8}},
     6:{'time':1,'cost':5,'output':{'product_D':1},'input':{'input_2':2, 'waste':1.2}},
}
waste=1000
processing_time=6000
raw_cost=6


sales_variables={
    'product_A':{'demand':5000,'price':18},
    'product_B':{'demand':1000,'price':24},
    'product_C':{'demand':10^9999,'price':11},
    'product_D':{'demand':10^9999,'price':11},
    
}


model.process_variables = Var(process_variables.keys(),domain=NonNegativeReals)



sales_revenue= sum(model.sales_variables[sales_var]*sales_variables[sales_var]['price']  for sales_var in sales_variables.keys())


process_cost= sum(model.process_variables[process_var]*process_variables[process_var]['cost']  for process_var in process_variables.keys())
total_process=sum(model.process_variables[process_var]  for process_var in process_variables.keys())

raw_amount=  model.process_variables[1]+model.process_variables[2]

procurement_cost=raw_amount*raw_cost

model.goal = Objective(expr = sales_revenue-process_cost-procurement_cost , sense=maximize)

model.constraints = ConstraintList()

model.constraints.add(total_process  <= processing_time )

for sales_var in sales_variables.keys():
    model.constraints.add( model.sales_variables[sales_var] -sales_variables[sales_var]['demand'] <= 0 )

produced_waste=model.process_variables[3] + 0.8*model.process_variables[4] - 0.8*model.process_variables[5] - 1.2*model.process_variables[6]    
model.constraints.add(  produced_waste  <= waste )   
  
model.constraints.add(   2*model.process_variables[1]  - 2*model.process_variables[3]  -  model.process_variables[4]-  2*model.process_variables[5]  == 0 ) 
model.constraints.add(   3*model.process_variables[2]  - 1*model.process_variables[3]  -  2*model.process_variables[4]-  2*model.process_variables[6]  == 0 )     
    
model.constraints.add(   model.process_variables[3] -  model.sales_variables['product_A']  == 0 )  
model.constraints.add(   model.process_variables[4] -  model.sales_variables['product_B']  == 0 ) 
model.constraints.add(   model.process_variables[5] -  model.sales_variables['product_C']  == 0 ) 
model.constraints.add(   model.process_variables[6] -  model.sales_variables['product_D']  == 0 ) 
    
model.pprint()

SolverFactory('cbc',executable=path).solve(model).write()

SolverFactory('glpk').solve(model).write()